In [ ]:
%run ./DFBRP_Simulator.ipynb

In [ ]:
BESTSOLTUIONKM = 0 
BESTSOLTUIONDEADHEAD = 1
BESTSOLTUIONREQUESTTIME = 2

import requests
from urllib3.util import Retry
from requests.adapters import HTTPAdapter
import pickle
import time as tm

# OSRM instance details
OSRM_HOST = "localhost"  # ROSRM server IP or hostname
OSRM_PORT = "5000"       # Port of OSRM server is listening on
SESSION = requests.Session()
RETRY = Retry(connect=5, read=5, backoff_factor=0.3)
ADAPTER = HTTPAdapter(max_retries=RETRY)
SESSION.mount('http://', ADAPTER)
SESSION.mount('https://', ADAPTER)

import copy

class MultiObjectiveTabuSearch:
    def __init__(self, numberOfVehicles=10, numberOfRequests=50, totalNumberOfIterations=75, totalNumberOfIterationsWithoutImprovement=10, threshold=20):
        self.renderingBusHandler = DFBRPSimulator(numberOfVehicles,numberOfRequests,render_mode="human")
        self.busHandler = DFBRPSimulator(numberOfVehicles,numberOfRequests)
        self.busHandler.requests = copy.deepcopy(self.renderingBusHandler.requests)
        self.busHandler.vehicles = copy.deepcopy(self.renderingBusHandler.vehicles)
        self.listOfActions = []
        self.tabuList = set()
        self.totalNumberOfIterations = totalNumberOfIterations
        self.totalNumberOfIterationsWithoutImprovement = totalNumberOfIterationsWithoutImprovement
        self.threshold = threshold

    def run(self):
        self.times = []
        end = False
        while end == False:
            startTime = tm.time()
            print("Current Request: " +str(self.busHandler.getCurrentRequest()))
            solutionKM = self._handleMultipleObjectiveDARP()
            self.busHandler = copy.deepcopy(solutionKM)
            self.renderingBusHandler.vehicles = copy.deepcopy(solutionKM.vehicles)
            self.renderingBusHandler.requests = copy.deepcopy(solutionKM.requests)

            
            #Advance the request to the next one and calculate the bus movements 
            self.busHandler.advanceToNextRequest()
            self.renderingBusHandler.advanceToNextRequest()
            if self.busHandler.endCheck():
                end = True
            self.busHandler.updateState()
            self.renderingBusHandler.updateState()

            listOfLayers = []
            for key in self.renderingBusHandler.layersDict:
                listOfLayers.append(self.renderingBusHandler.layersDict[key])
            for layer in self.renderingBusHandler.map.layers:
                if isinstance(layer, ipyleaflet.Marker) and layer not in listOfLayers:
                    self.renderingBusHandler.map.remove_layer(layer)

            for i,vehicle in enumerate(self.renderingBusHandler.vehicles):
                for cords in vehicle.routeList:
                    cordsMarker = Marker(location=(cords.getLatitude(),cords.getLongitude()),
                    icon=AwesomeIcon(name='map-marker', marker_color=self.renderingBusHandler.colors[i%10], icon_color="white", spin=False),
                    draggable=False)
                    self.renderingBusHandler.map.add_layer(cordsMarker)

            endTime = tm.time()

            if len(self.times) % 10 == 0:
                self.busHandler.times = self.times
                with open('ts_copy.pkl', 'wb') as f:
                    pickle.dump(self.busHandler, f)

            print("Request :" + str(self.busHandler.getCurrentRequest().getId())+" took to complete: " + str(endTime - startTime))
            print("+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=")
            self.times.append(endTime-startTime)
        print("Run Excetued Successfully")  

    def _getOSRMBetween2Points(self, cords1, cords2):
        url = f"http://{OSRM_HOST}:{OSRM_PORT}/route/v1/driving/{cords1.getLongitude()},{cords1.getLatitude()};{cords2.getLongitude()},{cords2.getLatitude()}?overview=full"
        response = SESSION.get(url)
        if response.status_code == 200:
            data = response.json()
            return data
    def _getClosestVehicles(self, request):
        vechDict = {}
        for vehicle in self.busHandler.vehicles:
            data = self._getOSRMBetween2Points(request.origin, vehicle.getPosition())
            vechDict[vehicle.getId()] = data["routes"][0]["distance"]
        
        #Sort the dictionary by value and turn it into a list of vehicle IDs
        sortedVehicles = sorted(vechDict.items(), key=lambda x: x[1])
        sortedVehicles = [x[0] for x in sortedVehicles]
        return sortedVehicles
    def _getTotalRequestTime(self, solution):
        totalRequestTime = 0
        for vehicle in solution.vehicles:
            totalRequestTime += vehicle.getRequestWaitingTime()
        return totalRequestTime
    def checkBrokenTimeConstraints(self, busHandler, vehicleIndex):
        vehicleRoute = busHandler.vehicles[vehicleIndex].getListOfCords()
        routeString = ""
        for cords in vehicleRoute:
            routeString += f"{cords.getLongitude()},{cords.getLatitude()};"
        routeString[:-1] #Remove Semi Colon (Final Character)
        url = f"http://{OSRM_HOST}:{OSRM_PORT}/route/v1/driving/{busHandler.vehicles[vehicleIndex].position.getLongitude()},{busHandler.vehicles[vehicleIndex].position.getLongitude()};{routeString}?overview=full"
        response = SESSION.get(url)
        if response.status_code == 200:
            data = response.json()

            #Loop through the data and check if the time constraints are broken anywhere 
            currentTime = busHandler.currentRequest.time
            for i,leg in enumerate(data["routes"][0]["legs"]):
                if vehicleRoute[i].getTimeWindow() > currentTime:
                    return True
                currentTime += (leg["distance"]/SPEED)
        return False
    def maintainRequestOrder(self, busHandler, vehicleIndex):
        vehicleRoute = busHandler.vehicles[vehicleIndex].routeList
        requestIds = set()
        for node in vehicleRoute:
            if node.getStart(): #If the node is a request origin, check if request IDs exist
                if node.getRequestId() in requestIds:
                    return False
            requestIds.add(node.getRequestId())
        return True
    def swapPositions(self,ls, pos1, pos2):
        # print("Swapping in Function: "+str(pos1)+" "+str(pos2)+" Route: "+str(ls))
        # print("Route at index: "+ str(pos1) + " " +str(ls[pos1]))
        # print("Route at index: "+ str(pos2) + " " +str(ls[pos2]))
        
        ls[pos1], ls[pos2] = ls[pos2], ls[pos1]
        return ls
    def getRequestPairs(self, solution, vehicleIndex):
        vehicleRoute = solution.vehicles[vehicleIndex].routeList
        requestPairs = {}
        for i,cord in enumerate(vehicleRoute):
            if cord.getStart():
                requestPairs[cord.getRequestId()] = [i]
            else:
                if cord.getRequestId() in requestPairs:
                    requestPairs[cord.getRequestId()].append(i)

        for key in list(requestPairs.keys()):
            if len(requestPairs[key]) != 2:
                raise Exception("Request Pair is not of length 2")

        return requestPairs
    def _removeRequestFromRoute(self, solution, vehicleIndex, requestId):
        vehicleRoute = solution.vehicles[vehicleIndex].routeList
        passengerAmount = solution.requests[requestId].getPassengerAmount()
        for i,cord in enumerate(vehicleRoute):
            if cord.getRequestId() == requestId:
                vehicleRoute.pop(i)
                break
        for i,cord in enumerate(vehicleRoute):
            if cord.getRequestId() == requestId:
                vehicleRoute.pop(i)
                break
        for reqId in list(solution.vehicles[vehicleIndex].stats.waitingReuqestList):
            if reqId == requestId:
                del solution.vehicles[vehicleIndex].stats.waitingReuqestList[reqId]
        solution.vehicles[vehicleIndex].currentLoad -= passengerAmount
        return solution
    def _addRequestToRoute(self, solution, vehicleIndex, requestId):
        passengerAmount = solution.requests[requestId].getPassengerAmount()
        solution.vehicles[vehicleIndex].currentLoad += passengerAmount
        solution.vehicles[vehicleIndex].stats.waitingReuqestList[requestId] = solution.requests[requestId]
        return solution
    def _handleMultipleObjectiveDARP(self):

        currentRequest = self.busHandler.getCurrentRequest()
        solutionFound, initialSolution = self._ARV(currentRequest)
        bestSolution = [initialSolution, initialSolution, initialSolution]
        if not solutionFound:
            self.listOfActions.append("REJECT")
            self.busHandler.rejectRequest()
            return self.busHandler
    
        solutionList = [initialSolution]
        historyOfSolutions = set()
        historyOfCanditates = set()
        historyOfCanditates.add(initialSolution)

        numberOfIterations = 0
        numberOfIterationsWithoutImprovement = 0
    
        while numberOfIterations < self.totalNumberOfIterations and numberOfIterationsWithoutImprovement < self.totalNumberOfIterationsWithoutImprovement and solutionList != []:
            currentSolution = solutionList.pop(0)
            candidateList = self._CLG(currentSolution)

            for candidate in candidateList:
                if candidate in historyOfCanditates:
                    candidateList.remove(candidate)
                else:
                    historyOfCanditates.add(candidate)

            for candidate in candidateList:
                for candidate2 in candidateList:
                    if candidate2.totalKMDistance <= candidate.totalKMDistance and candidate2.totalDeadHeadDistance <= candidate.totalDeadHeadDistance and self._getTotalRequestTime(candidate2) <= self._getTotalRequestTime(candidate):
                        if candidate2.totalKMDistance < candidate.totalKMDistance or candidate2.totalDeadHeadDistance < candidate.totalDeadHeadDistance or self._getTotalRequestTime(candidate2) < self._getTotalRequestTime(candidate):
                            candidateList.remove(candidate)
                            break

            if candidateList == []:
                numberOfIterationsWithoutImprovement += 1
                print("No non dominated candidate found")
                continue
        
            if len(candidateList) > self.threshold:
                n = self.threshold//3
                
                orderedCandidateListKM = sorted(candidateList, key=lambda x: (x.totalKMDistance))
                orderedCandidateListDeadHead = sorted(candidateList, key=lambda x: (x.totalDeadHeadDistance))
                orderedCandidateListVehicleUsage = sorted(candidateList, key=lambda x: (self._getTotalRequestTime(x)))

                newNeighbourhood = orderedCandidateListKM[:n]
                i,j = 0,0
                while i < n:
                    if orderedCandidateListDeadHead[j] not in newNeighbourhood:
                        newNeighbourhood.append(orderedCandidateListDeadHead[j])
                        i += 1
                        j += 1
                    else:
                        j += 1
                i, j = 0, 0
                while i < n:
                    if orderedCandidateListVehicleUsage[j] not in newNeighbourhood:
                        newNeighbourhood.append(orderedCandidateListVehicleUsage[j])
                        i += 1
                        j += 1
                    else:
                        j += 1
                
                for solution in newNeighbourhood:
                    if solution not in historyOfSolutions:
                        solutionList.append(solution)
                        historyOfSolutions.add(solution)
            else:
                for solution in candidateList:
                    if solution not in historyOfSolutions:
                        solutionList.append(solution)
                        historyOfSolutions.add(solution)
            
            #Increment the number of iterations
            numberOfIterations += 1
            improvement = False

            for neighbourHood in solutionList:
                if neighbourHood.totalKMDistance < bestSolution[BESTSOLTUIONKM].totalKMDistance:
                    bestSolution[BESTSOLTUIONKM] = neighbourHood
                    improvement = True
                if neighbourHood.totalDeadHeadDistance < bestSolution[BESTSOLTUIONDEADHEAD].totalDeadHeadDistance:
                    bestSolution[BESTSOLTUIONDEADHEAD] = neighbourHood
                    improvement = True
                if self._getTotalRequestTime(neighbourHood) < self._getTotalRequestTime(bestSolution[BESTSOLTUIONREQUESTTIME]):
                    bestSolution[BESTSOLTUIONREQUESTTIME] = neighbourHood
                    improvement = True
            if improvement:
                numberOfIterationsWithoutImprovement = 0
            else:
                numberOfIterationsWithoutImprovement += 1
        
        return bestSolution[BESTSOLTUIONKM]

    #Find initial Solution
    def _ARV(self,request):
        orderedListOfClosestVehicles = self._getClosestVehicles(request)
        for vehicle in orderedListOfClosestVehicles:
            solutionFound, solution = self._ASRSV(request, vehicle)
            if solutionFound:
                #Compute the totalKM distance, the deadHeading disantance and vehicleUsage
                solution.totalKMDistance = solution.getTotalDistance()
                solution.totalDeadHeadDistance = solution.getTotaLDeadHeadDistance()
                solution.totalRequestTime = self._getTotalRequestTime(solution)
                return solutionFound, solution
        return False, None
    def _ASRSV(self,request, vehicle):
        #Check that the capacity contraitns of the bus are not broken
        if self.busHandler.vehicles[vehicle].getRouteSize() + request.getPassengerAmount() > self.busHandler.vehicles[vehicle].getCapacity():
            return False, None

        #Create a deep copy of the busHandler class to check for time contraints
        copyBusHandler = copy.deepcopy(self.busHandler)

        #Get length of the route
        routeLength = copyBusHandler.vehicles[vehicle].getRouteSize()
        copyBusHandler.acceptRequest(vehicle,routeLength,routeLength+1)

        #Check for broken time contriansts on that vehicle: 
        if self.checkBrokenTimeConstraints(copyBusHandler,vehicle):
            return False, None
        
        #If time contraisnts are not broken check if the request origin can be moved towards the beginning
        broken = False
        latestValid = copy.deepcopy(copyBusHandler)

        while not broken:
            currentOriginIndex = copyBusHandler.vehicles[vehicle].routeList.index(request.origin)
            
            #If made it to the start of the route
            if currentOriginIndex <= 0:
                break

            #Swap the origin with the previous node
            # print("Swapping: "+str(currentOriginIndex)+" "+str(currentOriginIndex-1)+" Route: "+str(copyBusHandler.vehicles[vehicle].routeList))
            # print("Route at index: "+ str(currentOriginIndex) + " " +str(copyBusHandler.vehicles[vehicle].routeList[currentOriginIndex]))
            # print("Route at index: "+ str(currentOriginIndex-1) + " " +str(copyBusHandler.vehicles[vehicle].routeList[currentOriginIndex-1]))
            copyBusHandler.vehicles[vehicle].routeList = self.swapPositions(copyBusHandler.vehicles[vehicle].routeList, currentOriginIndex, currentOriginIndex-1)

            #Check if the time constraints are broken
            if self.checkBrokenTimeConstraints(copyBusHandler,vehicle):
                broken = True
            else:
                latestValid = copy.deepcopy(copyBusHandler)
        return True, latestValid
    def _CLG(self, solution):
        candidateList = []
        #Check all swap exchanges for the current request
        for vehicle in solution.vehicles:
            #If route only has 1 or less points skip 
            if vehicle.getRouteSize() <= 1:
                continue
        
            # print(f"Vehicle: {vehicle.getId()} RouteSize: {vehicle.getRouteSize()}")
            # print(f"Route Len: {len(vehicle.route.routeList.getListOfCords())}")

            for i in range(vehicle.getRouteSize()):
                for j in range(i,vehicle.getRouteSize()):
                    if i == j: # Skip if the two indexes are the same
                        continue

                    solutionCopy = copy.deepcopy(solution)
                    #Swap the two indexes

                    solutionCopy.vehicles[vehicle.getId()].routeList = self.swapPositions(solutionCopy.vehicles[vehicle.getId()].routeList,i,j)

                    #Check if the time constraints are broken
                    if self.checkBrokenTimeConstraints(solutionCopy,vehicle.getId()):
                        continue
                    
                    #Check if the solution has maintained the order constraints 
                    if not self.maintainRequestOrder(solutionCopy,vehicle.getId()):
                        continue
                    
                    solutionCopy.totalKMDistance = solutionCopy.getTotalDistance()
                    solutionCopy.totalDeadHeadDistance = solutionCopy.getTotaLDeadHeadDistance()
                    solutionCopy.totalRequestTime = self._getTotalRequestTime(solutionCopy)
                    candidateList.append(solutionCopy)
                
        #Check if request pairs can be moved to the beginning or end of the route
        requestPairs = self.getRequestPairs(solution,vehicle.getId())
        route = solution.vehicles[vehicle.getId()].routeList
        for requestId in requestPairs:
            for vehicle2 in solution.vehicles:
                if vehicle == vehicle2:
                    continue
                
                request = solution.requests[requestId]
                #Check if the capacity contraints of the bus are not broken
                if solution.vehicles[vehicle2.getId()].getRouteSize() + request.getPassengerAmount() > solution.vehicles[vehicle2.getId()].getCapacity():
                    continue
                
                solutionCopy = copy.deepcopy(solution)
                solutionCopy.vehicles[vehicle2.getId()].routeList.insert(0,route[requestPairs[requestId][1]])
                solutionCopy.vehicles[vehicle2.getId()].routeList.insert(0,route[requestPairs[requestId][0]])
                
                if not self.checkBrokenTimeConstraints(solutionCopy,vehicle2.getId()):
                    solutionCopy = self._removeRequestFromRoute(solutionCopy,vehicle.getId(),requestId)
                    solutionCopy = self._addRequestToRoute(solutionCopy,vehicle2.getId(),requestId)
                    solutionCopy.totalKMDistance = solutionCopy.getTotalDistance()
                    solutionCopy.totalDeadHeadDistance = solutionCopy.getTotaLDeadHeadDistance()
                    solutionCopy.totalRequestTime = self._getTotalRequestTime(solutionCopy)
                    candidateList.append(solutionCopy)
                
                if len(solution.vehicles[vehicle2.getId()].routeList) == 0: #Inserting at start at end will result in the same solution
                    continue
            
                solutionCopy2 = copy.deepcopy(solution)
                solutionCopy2.vehicles[vehicle2.getId()].routeList.append(route[requestPairs[requestId][0]])
                solutionCopy2.vehicles[vehicle2.getId()].routeList.append(route[requestPairs[requestId][1]])
                if not self.checkBrokenTimeConstraints(solutionCopy2,vehicle2.getId()):
                    solutionCopy2 = self._removeRequestFromRoute(solutionCopy2,vehicle.getId(),requestId)
                    solutionCopy2 = self._addRequestToRoute(solutionCopy2,vehicle2.getId(),requestId)
                    solutionCopy2.totalKMDistance = solutionCopy2.getTotalDistance()
                    solutionCopy2.totalDeadHeadDistance = solutionCopy2.getTotaLDeadHeadDistance()
                    solutionCopy2.totalRequestTime = self._getTotalRequestTime(solutionCopy2)
                    candidateList.append(solutionCopy2)
        return candidateList

<h3>Running with Different Parameters</h3>

In [ ]:
T1 = MultiObjectiveTabuSearch(numberOfVehicles=5, numberOfRequests=50)
T2 = MultiObjectiveTabuSearch(numberOfVehicles=5, numberOfRequests=150)
T3 = MultiObjectiveTabuSearch(numberOfVehicles=5, numberOfRequests=500)
T4 = MultiObjectiveTabuSearch(numberOfVehicles=5, numberOfRequests=1000)
T5 = MultiObjectiveTabuSearch(numberOfVehicles=10, numberOfRequests=50)
T6 = MultiObjectiveTabuSearch(numberOfVehicles=10, numberOfRequests=150)
T7 = MultiObjectiveTabuSearch(numberOfVehicles=10, numberOfRequests=500)
T8 = MultiObjectiveTabuSearch(numberOfVehicles=10, numberOfRequests=1000)
T9 = MultiObjectiveTabuSearch(numberOfVehicles=20, numberOfRequests=50)
T10 = MultiObjectiveTabuSearch(numberOfVehicles=20, numberOfRequests=150)
T11 = MultiObjectiveTabuSearch(numberOfVehicles=20, numberOfRequests=500)
T12 = MultiObjectiveTabuSearch(numberOfVehicles=20, numberOfRequests=1000)

In [ ]:
import pickle
import copy
r1 = pickle.load(open("EvalCases/r1.pkl","rb"))
r2 = pickle.load(open("EvalCases/r2.pkl","rb"))
r3 = pickle.load(open("EvalCases/r3.pkl","rb"))
r4 = pickle.load(open("EvalCases/r4.pkl","rb"))

T1.busHandler.requests = copy.deepcopy(r1)
T1.renderingBusHandler.requests = copy.deepcopy(r1)
T2.busHandler.requests = copy.deepcopy(r2)
T2.renderingBusHandler.requests = copy.deepcopy(r2)
T3.busHandler.requests = copy.deepcopy(r3)
T3.renderingBusHandler.requests = copy.deepcopy(r3)
T4.busHandler.requests = copy.deepcopy(r4)
T4.renderingBusHandler.requests = copy.deepcopy(r4)
T5.busHandler.requests = copy.deepcopy(r1)
T5.renderingBusHandler.requests = copy.deepcopy(r1)
T6.busHandler.requests = copy.deepcopy(r2)
T6.renderingBusHandler.requests = copy.deepcopy(r2)
T7.busHandler.requests = copy.deepcopy(r3)
T7.renderingBusHandler.requests = copy.deepcopy(r3)
T8.busHandler.requests = copy.deepcopy(r4)
T8.renderingBusHandler.requests = copy.deepcopy(r4)
T9.busHandler.requests = copy.deepcopy(r1)
T9.renderingBusHandler.requests = copy.deepcopy(r1)
T10.busHandler.requests = copy.deepcopy(r2)
T10.renderingBusHandler.requests = copy.deepcopy(r2)
T11.busHandler.requests = copy.deepcopy(r3)
T11.renderingBusHandler.requests = copy.deepcopy(r3)
T12.busHandler.requests = copy.deepcopy(r4)
T12.renderingBusHandler.requests = copy.deepcopy(r4)

In [ ]:
print("Running T1")
T1.run()
T1.renderingBusHandler = None
pickle.dump(T1,open("EvalCases/T1_TS.pkl","wb"))

In [ ]:
print("Running T2")
T2.run()
T2.renderingBusHandler = None
pickle.dump(T2,open("EvalCases/T2_TS.pkl","wb"))

In [ ]:
print("Running T3")
T3.run()
T3.renderingBusHandler = None
pickle.dump(T3,open("EvalCases/T3_TS.pkl","wb"))

In [ ]:
print("Running T4")
T4.run()
T4.renderingBusHandler = None
pickle.dump(T4,open("EvalCases/T4_TS.pkl","wb"))

In [ ]:
print("Running T5")
T5.run()
T5.renderingBusHandler = None
pickle.dump(T5,open("EvalCases/T5_TS.pkl","wb"))

In [ ]:
print("Running T6")
T6.run()
T6.renderingBusHandler = None
pickle.dump(T6,open("EvalCases/T6_TS.pkl","wb"))

In [ ]:
print("Running T7")
T7.run()
T7.renderingBusHandler = None
pickle.dump(T7,open("EvalCases/T7_TS.pkl","wb"))

In [ ]:
print("Running T8")
T8.run()
T8.renderingBusHandler = None
pickle.dump(T8.requests,open("EvalCases/T8_TS.pkl","wb"))

In [ ]:
print("Running T9")
T9.run()
T9.renderingBusHandler = None
pickle.dump(T9,open("EvalCases/T9_TS.pkl","wb"))

In [ ]:
print("Running T10")
T10.run()
T10.renderingBusHandler = None
pickle.dump(T10,open("EvalCases/T10_TS.pkl","wb"))

In [ ]:
print("Running T11")
T11.run()
T11.renderingBusHandler = None
pickle.dump(T11,open("EvalCases/T11_TS.pkl","wb"))

In [ ]:
print("Running T12")
T12.run()
T12.renderingBusHandler = None
pickle.dump(T12,open("EvalCases/T12_TS.pkl","wb"))

<h3>Tabu Search Testing</h3>

In [ ]:
# import time as ts
# times = divide_time_interval(50)
# # newTestEnv = MultiObjectiveTabuSearch(numberOfVehicles=10,numberOfRequests=50)
# d1 = MultiObjectiveTabuSearch(2,50)
# d1.busHandler.requests[0] = Request(1, Cords(35.8997,14.5146), Cords(35.85232, 14.38536), times[0], 1)
# d1.busHandler.currentRequest = d1.busHandler.requests[0]

# solutionKM = d1._handleMultipleObjectiveDARP()

# d1.busHandler = copy.deepcopy(solutionKM)
# d1.renderingBusHandler.vehicles = copy.deepcopy(solutionKM.vehicles)
# d1.renderingBusHandler.requests = copy.deepcopy(solutionKM.requests)

# listCandidates = d1._CLG(d1.busHandler)
# print("Number of Candidates: "+str(len(listCandidates)))


# #Advance the request to the next one and calculate the bus movements 
# d1.busHandler.advanceToNextRequest()
# d1.renderingBusHandler.advanceToNextRequest()
# d1.busHandler.updateState()
# d1.renderingBusHandler.updateState()